In [1]:
import pandas as pd

In [2]:
def read_and_give_dict(path):
    """
        The function returns a Dictionary {"Version", "Defination", "Sequence"}
        for the protiens in a file, given in the path.
        """
    with open(path) as file:
        info = file.read()
    info = info.split(">")
    while True:
        try:
            info.remove("")
        except:
            break        
    protien_list, protien = [], {"Version": [], "Defination": [], "Sequence": []}
    for i in info:
        temp, seq = i.split("\n", maxsplit=1)
        protien['Version'].append(temp.split(" ", maxsplit=1)[0])
        protien['Defination'].append(temp.split(" ", maxsplit=1)[1])
        protien["Sequence"].append("".join(seq.split("\n")))
    return protien

In [3]:
protien = read_and_give_dict("./../Protein_alignment.txt")
len(protien['Version'][0])

10

In [4]:
"""
    Importing the scoring scheme "BLOSUM62.txt", to score the 
    simliarities between the various protiens to be used in
    making the Phylogenic Tree
    """
with open("./../BLOSUM62.txt") as file:
    info = file.read()
info = info.split("\n")
info = [ x.split() for x in info ]
scores = pd.DataFrame(info)
temp = list(scores.iloc[0])
temp[0] = "Remove"
scores.columns, scores.index = temp, temp
scores = scores.drop(scores.index[0])
scores = scores.drop(scores.columns[0], axis=1)
scores.head()

,A,R,N,D,C,Q,E,G,H,I,...,P,S,T,W,Y,V,B,Z,X,*
A,4,-1,-2,-2,0,-1,-1,0,-2,-1,...,-1,1,0,-3,-2,0,-2,-1,0,-4
R,-1,5,0,-2,-3,1,0,-2,0,-3,...,-2,-1,-1,-3,-2,-3,-1,0,-1,-4
N,-2,0,6,1,-3,0,0,0,1,-3,...,-2,1,0,-4,-2,-3,3,0,-1,-4
D,-2,-2,1,6,-3,0,2,-1,-1,-3,...,-1,0,-1,-4,-3,-3,4,1,-1,-4
C,0,-3,-3,-3,9,-3,-4,-3,-3,-1,...,-3,-1,-1,-2,-2,-1,-3,-3,-2,-4


In [5]:
def get_score(str1, str2):
    global scores
    m = len(str2)
    str1 = "*".join( str1.split("-") )
    str2 = "*".join( str2.split("-") )
    score = 0
    for i in range(m):
        score += int(scores.at[str1[i], str2[j]])
    return score

In [6]:
distance_matrix = []
for i in range(len(protien['Sequence'])):
    distance = []
    for j in range(len(protien['Sequence'])):
        distance.append( get_score(protien['Sequence'][i], protien['Sequence'][j]) )
    distance_matrix.append(distance)

In [7]:
df = pd.DataFrame(distance_matrix)
names = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E",
         5: "F", 6:"G", 7:"H", 8:"I", 9:"J"}
df.rename(names, inplace=True)
df.columns = list(names.values())
df

,A,B,C,D,E,F,G,H,I,J
A,-85,-56,-121,-260,-98,-153,-121,-121,-170,-121
B,-61,-69,-101,-244,-89,-128,-101,-101,-186,-101
C,-86,-67,-118,-250,-101,-132,-118,-118,-182,-118
D,-63,-66,-94,-238,-96,-126,-94,-94,-160,-94
E,-80,-40,-101,-241,-99,-137,-101,-101,-173,-101
F,-100,-46,-126,-249,-114,-146,-126,-126,-169,-126
G,-63,-37,-82,-262,-93,-132,-82,-82,-184,-82
H,-61,-45,-82,-239,-92,-125,-82,-82,-179,-82
I,-67,-49,-92,-235,-94,-137,-92,-92,-179,-92
J,-65,-56,-91,-235,-93,-139,-91,-91,-180,-91


In [8]:
df.to_csv("Pdistance.txt")